In [7]:
import glob
import re
import pathlib
import json
import os
import shutil

from tqdm.auto import tqdm

In [8]:
cainfo = glob.glob('../../../models/Lora/*.civitai.info')

base_model_count = {}

for cai in tqdm(cainfo):
    fname = pathlib.Path(cai).stem.split('.civitai')[0]
    with open(cai, 'r') as f:
        bm = json.load(f)['baseModel']
    res = glob.glob(f'../../../models/Lora/{fname}*')
    for r in res:
        dest_fname = os.path.basename(r)
        dest_file = os.path.join(f'../../../models/Lora/{bm}', dest_fname)
        shutil.move(r, dest_file)
        print(r, '->', dest_file)

0it [00:00, ?it/s]

In [9]:
def read_metadata_from_safetensors(filename):
    with open(filename, mode="rb") as file:
        metadata_len = file.read(8)
        metadata_len = int.from_bytes(metadata_len, "little")
        json_start = file.read(2)

        assert metadata_len > 2 and json_start in (b'{"', b"{'"), f"{filename} is not a safetensors file"

        res = {}

        try:
            json_data = json_start + file.read(metadata_len-2)
            json_obj = json.loads(json_data)
            for k, v in json_obj.get("__metadata__", {}).items():
                res[k] = v
                if isinstance(v, str) and v[0:1] == '{':
                    try:
                        res[k] = json.loads(v)
                    except Exception:
                        pass
        except Exception:
             Exception(f"Error reading metadata from file: {filename}")

        return res

In [10]:
files = glob.glob('../wildcards/**/*.txt', recursive=True) + glob.glob('../wildcards/**/*.yaml', recursive=True)

pattern = re.compile('<lora:(.+?):')

In [11]:
matches = []
for file in files:
    with open(file, 'r') as f:
        matches += [ x.strip() for x in pattern.findall(f.read()) ]
matches = set(matches)

In [12]:
loras = glob.glob('../../../models/Lora/**/*.safetensors', recursive=True)

ss_names = []
fnames = []
lora_dict = {
    "forward": {},
    "reverse": {}
}

for lora in tqdm(loras):
    fname = pathlib.Path(lora).stem
    fnames.append(fname)
    try:
        ss_name = read_metadata_from_safetensors(lora)['ss_output_name']
    except:
        ss_name = fname

    ss_name = ss_name.strip()

    baseModel = os.path.normpath(lora).split('Lora')[1].split(os.sep)[1]
    lora_dict['forward'][fname] = {
                                   'ss_name': ss_name,
                                   'baseModel': baseModel
                                   }

    if ss_name in lora_dict['reverse'].keys():
        lora_dict['reverse'][ss_name] += [fname]
    else:
        lora_dict['reverse'][ss_name] = [fname]
    ss_names.append(ss_name)
ss_names = set(ss_names)
fnames = set(fnames)
with open('../wildcards/used_resources.json', 'w') as f:
    json.dump(lora_dict, f, indent=4)

  0%|          | 0/2550 [00:00<?, ?it/s]

In [13]:
matches.difference(ss_names)

{'1990sPCstyle_IL_v4',
 '2Foxy_-_fnaf_thegeckoninja_style',
 'AlcinaDimitrescuxl-12-153',
 'AlcinaDimitrescuxl-12v6',
 'Aphrodite_Fortnite-000005',
 'Bowsette-2',
 'CHARACTER_Machamp',
 'CHARACTER_Machoke',
 'CrystalMommy',
 'Dmon ill-000045',
 'ErisaFNIlluV1',
 'FN_Katalina',
 'Garuda',
 'IL_Sarah_Damon',
 'KDA_Kaisa_PONY',
 'Katt_Monroe_illu',
 'Lia_Venegas_v2',
 'LiliT8-07',
 'MatchaXL',
 'Mervamon_-_Digimon-000009',
 'Mervamon_-_Digimon-000009(1)',
 'Mervamon_-_Digimon-000009(1.5.2)',
 'MilaDOA-10',
 'Monster Girl Dragon',
 'POKEMON_MEOWSTIC_ FEMALE',
 'POKEMON_MEOWSTIC_MALE',
 'PONY_Sarah_Damon',
 'Pauline-1',
 'Princess_Elise_-_Sonic_06_Illustrious',
 'Protogen-base',
 'Protogen-ponyxl',
 'Renamon iLLus',
 'Riley_Andersen_A',
 'SerahFarron',
 'Shiva',
 'SmiteAphroditeIllu',
 'Stella',
 'Venom Absorb',
 'YunaPony',
 '[Bethesda] Flame Atronach (Skyrim) Character Illustrious',
 "[DAGASI (Sega Sonic)] Miles 'Tails' Prower Character 2.0 IllustriousXL",
 "[DAGASI (Sega Sonic)] Miles 'T

In [14]:
matches.difference(ss_names.union(fnames))

set()

In [15]:
for x in lora_dict['reverse']:
    if x in matches:
        if len(lora_dict['reverse'][x]) > 1:
            print(x, lora_dict['reverse'][x])

Foxy_-_fnaf_thegeckoninja_style ['2Foxy_-_fnaf_thegeckoninja_style', 'Foxy_-_fnaf_thegeckoninja_style']
Aphrodite_Fortnite ['Aphrodite_Fortnite-000005', 'Aphrodite_Fortnite']
Bowsette ['Bowsette', 'Bowsette-2', 'Bowsette']
br hatsune miku ['br hatsune miku_v2', 'br hatsune miku']
Katt_Monroe ['Katt_Monroe_illu', 'Katt_Monroe']
KDA_Kaisa ['KDA_Kaisa', 'KDA_Kaisa_PONY']
Lucky_Chloe_Tekken_PONY_XL_by_UOC ['Lucky_Chloe_Tekken_PONY_XL_by_UOC', 'Lucky_Chloe_Tekken_PONY_XL_by_UOC']
TimeKeeperXL ['MatchaXL', 'TimeKeeperXL']
Princess_Elise_-_Sonic_06 ['Princess_Elise_-_Sonic_06_Illustrious', 'Princess_Elise_-_Sonic_06']
Princess_Peach ['Princess_Peach', 'Princess_Peach_Pony']
SmiteAphrodite ['SmiteAphroditeIllu', 'SmiteAphrodite']
MilaDOA ['MilaDOA-10', 'MilaDOA']
nemia ['nemia-10', 'nemia']
Protogen ['Protogen-base', 'Protogen-ponyxl']
Riley_Andersen ['Riley_Andersen', 'Riley_Andersen_A']
